In [1]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install wget
!pip install docker
!pip install python-magic
!pip install elasticsearch_dsl
!pip install fire
!pip install unidecode
!pip install fitz
!pip install frontend
!pip install pymupdf
!pip install arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0


In [2]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline, pipeline
from huggingface_hub import HfFolder, notebook_login
import wget
import os
import pandas as pd
from datasets import Dataset, load_dataset
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import arxiv
import re

In [3]:
# Set pandas option to display the full content of each column
pd.set_option('display.max_colwidth', None)

In [32]:
# !unzip axcell.zip

In [4]:
#Logging in into Hugging Face
notebook_login()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
ARXIV_PAPERS_URL = '/content/arxiv-papers.csv'

In [55]:
def read_arxiv_papers(path):
    return pd.read_csv(path)

In [37]:
arxiv_papers = read_arxiv_papers(ARXIV_PAPERS_URL)

print(f'Number of papers:           {len(arxiv_papers):8}')
print(f'└── with LaTeX source:      {(~arxiv_papers.status.isin(["no-tex", "withdrawn"])).sum():8}')
print(f'Number of extracted tables: {arxiv_papers.tables.sum():8}')

Number of papers:             104710
└── with LaTeX source:         93811
Number of extracted tables:   277946


In [38]:
# def get_eprint_link(paper):
#     return f'https://arxiv.org/e-print/{paper.arxiv_id}'

# arxiv_papers['Links'] = arxiv_papers.apply(get_eprint_link, axis=1)
arxiv_papers.drop(['sha256', 'status'], axis=1, inplace=True)
arxiv_papers.head(5)

,arxiv_id,archive_size,title,sections,tables
0,0704.0004v1,9486,A determinant of Stirling cycle numbers counts unlabeled acyclic single-source automata,1,0
1,0704.0010v1,45695,"Partial cubes: structures, characterizations, and constructions",0,0
2,0704.0012v1,9560,Distribution of integral Fourier Coefficients of a Modular Form of Half Integral Weight Modulo Primes,7,0
3,0704.0858v1,355195,Lessons Learned from the deployment of a high-interaction honeypot,0,0
4,0704.0861v1,693752,Empirical analysis and statistical modeling of attack processes based on honeypots,0,0


In [39]:
arxiv_papers.shape[0]

104710

In [40]:
# Function to extract abstract using arxiv module
def extract_abstract(arxiv_id):
    search = arxiv.Search(id_list=[arxiv_id])
    for result in search.results():
        return result.summary

In [41]:
# Define a function that generates the prompt text for each row
def generate_prompt(row):
    return f"Generate a concise 150-250 word abstract for the work titled '{row['title']}' in a single paragraph. \
    Focus on describing the results, the problem addressed, and the key ideas and methodologies. \
    Use formal academic and scientific language in the first-person plural and active voice. \
    Only provide the abstract and strictly exclude any additional text or commentary. \
    Strictly DO NOT add the title of the work at the beginning of the generated abstract. \
    Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. \
    Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. \
    Strictly GIVE ONLY ONE paragraph as generated abstract."

# Apply the function to each row to create the new 'Prompt' column
arxiv_papers['Prompt'] = arxiv_papers.apply(generate_prompt, axis=1)

arxiv_papers.head(5)

,arxiv_id,archive_size,title,sections,tables,Prompt
0,0704.0004v1,9486,A determinant of Stirling cycle numbers counts unlabeled acyclic single-source automata,1,0,"Generate a concise 150-250 word abstract for the work titled 'A determinant of Stirling cycle numbers counts unlabeled acyclic single-source automata' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract."
1,0704.0010v1,45695,"Partial cubes: structures, characterizations, and constructions",0,0,"Generate a concise 150-250 word abstract for the work titled 'Partial cubes: structures, characterizations, and constructions' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract."
2,0704.0012v1,9560,Distribution of integral Fourier Coefficients of a Modular Form of Half Integral Weight Modulo Primes,7,0,"Generate a concise 150-250 word abstract for the work titled 'Distribution of integral Fourier Coefficients of a Modular Form of Half Integral Weight Modulo Primes' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract."
3,0704.0858v1,355195,Lessons Learned from the deployment of a high-interaction honeypot,0,0,"Generate a concise 150-250 word abstract for the work titled 'Lessons Learned from the deployment of a high-interaction honeypot' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract."
4,0704.0861v1,693752,Empirical analysis and statistical modeling of attack processes based on honeypots,0,0,"Generate a concise 150-250 word abstract for the work titled 'Empirical analysis and statistical modeling of attack processes based on honeypots' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Us

In [7]:
# Set row count limit
row_count = 3000
save_count = 50

In [8]:
if not os.path.exists('/content/drive/My Drive/arxiv_papers/'):
    os.makedirs('/content/drive/My Drive/arxiv_papers/')

In [44]:
# Add new column for abstracts
arxiv_papers['Abstract'] = ''

# Process each row and save to CSV every 50 rows
for index, row in arxiv_papers.iterrows():
    if index >= row_count:
        break
    arxiv_papers.at[index, 'Abstract'] = extract_abstract(row['arxiv_id'])

    if (index + 1) % save_count == 0:
        arxiv_papers.to_csv('/content/drive/My Drive/arxiv_papers/org_arxiv_papers.csv', index=False)
        print(f'Saved human-written abstract at row {index + 1}')

# Save the final DataFrame
arxiv_papers.to_csv('/content/drive/My Drive/org_arxiv_papers.csv', index=False)

<ipython-input-40-53e660faa2cb>:4: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Saved human-written abstract at row 50
Saved human-written abstract at row 100
Saved human-written abstract at row 150
Saved human-written abstract at row 200
Saved human-written abstract at row 250
Saved human-written abstract at row 300
Saved human-written abstract at row 350
Saved human-written abstract at row 400
Saved human-written abstract at row 450
Saved human-written abstract at row 500
Saved human-written abstract at row 550
Saved human-written abstract at row 600
Saved human-written abstract at row 650
Saved human-written abstract at row 700
Saved human-written abstract at row 750
Saved human-written abstract at row 800
Saved human-written abstract at row 850
Saved human-written abstract at row 900
Saved human-written abstract at row 950
Saved human-written abstract at row 1000
Saved human-written abstract at row 1050
Saved human-written abstract at row 1100
Saved human-written abstract at row 1150
Saved human-written abstract at row 1200
Saved human-written abstract at row 

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [10]:
# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use bfloat16 to reduce memory usage
    device_map="auto"  # Automatically use the GPU if available
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [47]:
arxiv_papers.shape[0]

104710

In [48]:
arxiv_papers = arxiv_papers[arxiv_papers.Abstract != '']

In [49]:
arxiv_papers.shape[0]

3000

In [50]:
arxiv_papers['Prompt'].head(2)

0    Generate a concise 150-250 word abstract for the work titled 'A determinant of Stirling cycle numbers counts unlabeled acyclic single-source automata' in a single paragraph.     Focus on describing the results, the problem addressed, and the key ideas and methodologies.     Use formal academic and scientific language in the first-person plural and active voice.     Only provide the abstract and strictly exclude any additional text or commentary.     Strictly DO NOT add the title of the work at the beginning of the generated abstract.     Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract.     Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract.     Strictly GIVE ONLY ONE paragraph as generated abstract.
1                            Generate a concise 150-250 word abstract for the work titled 'Partial cubes: structures, characterizations, and constructions' in a single para

In [11]:
def generate_text(prompt):
    # Encode the prompt into tokens and send to the same device as the model
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Generate text responses with specified parameters
    outputs = model.generate(**inputs, max_new_tokens=300, num_return_sequences=1, temperature=0.7)

    # Decode generated tokens to strings
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt text from the output by slicing the tokens
    prompt_len = len(tokenizer.encode(prompt))
    clean_generated_text = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

    # Additional cleanup using regular expressions to remove common trailing notes or requests
    abstract_text = re.sub(r'(\.\s+Please note.*|\.\s+Best regards.*)', '.', clean_generated_text, flags=re.DOTALL)

    # Clean up the start to ensure no leading punctuation or whitespace
    abstract_text = re.sub(r'^[\.\s]+', '', abstract_text)

    # Define a list of patterns to remove from the text
    patterns_to_remove = [
        r'150-250 words\. 1st person plural, active voice, formal academic language\. No additional text or commentary\. Abstract only\. Please ensure the abstract is concise and within the specified word limit\. Ensure the abstract is written in.*$',
        r'Use formal academic and scientific language in the first-person plural and active voice\.$',
        r'Focus on describing the results, the problem addressed, and the key ideas and methodologies\.$',
        r'150-250 words.',
        r'Abstract.',
        r'First-person plural.',
        r'first-person plural.',
        r'1st person plural',
        r'Active voice.',
        r'active voice.',
        r'Formal academic and scientific language.',
        r'No additional text or commentary.',
        r'Single paragraph.',
        r'Focus on describing results, problem addressed, and key',
        r'Here is the generated abstract:',
        r'Here is the abstract: '
        # Add more patterns as needed
    ]

    # Remove each pattern from the abstract_text
    for pattern in patterns_to_remove:
        abstract_text = re.sub(pattern, '', abstract_text, flags=re.DOTALL)

    return abstract_text



In [12]:
# Check and create directory on Google Drive
save_directory = '/content/drive/My Drive/arxiv_papers'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
save_path = os.path.join(save_directory, 'arxiv_papers_with_MT.csv')

In [54]:
# Process DataFrame and save every 50 rows
for index, row in arxiv_papers.iterrows():
    arxiv_papers.at[index, 'machine_text'] = generate_text(row['Prompt'])
    if (index + 1) % save_count == 0:
        arxiv_papers.to_csv(save_path, index=False)
        print(f'Saved progress at row {index + 1}')

# Save the final result
arxiv_papers.to_csv(save_path, index=False)
print('Final save completed.')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [13]:
ARXIV_PAPERS_URL2 = '/content/drive/My Drive/arxiv_papers/org_arxiv_papers.csv'
def read_arxiv_papers2(path):
    return pd.read_csv(path)


arxiv_papers2 = read_arxiv_papers2(ARXIV_PAPERS_URL2)

In [14]:
arxiv_papers2.shape[0]

104710

In [15]:
arxiv_papers2.tail(5)

,arxiv_id,archive_size,title,sections,tables,Prompt,Abstract
104705,2002.08204v1,346382,SYMOG: learning symmetric mixture of Gaussian modes for improved fixed-point quantization,0,0,"Generate a concise 150-250 word abstract for the work titled 'SYMOG: learning symmetric mixture of Gaussian modes for improved fixed-point quantization' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract.",NaN
104706,2002.08253v1,89874,Distance-Based Regularisation of Deep Networks for Fine-Tuning,0,0,"Generate a concise 150-250 word abstract for the work titled 'Distance-Based Regularisation of Deep Networks for Fine-Tuning' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract.",NaN
104707,2002.08264v1,293953,Molecule Attention Transformer,18,20,"Generate a concise 150-250 word abstract for the work titled 'Molecule Attention Transformer' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract.",NaN
104708,2002.08301v1,2000723,Multi-wavelet residual dense convolutional neural network for image denoising,17,3,"Generate a concise 150-250 word abstract for the work titled 'Multi-wavelet residual dense convolutional neural network for image denoising' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide the abstract and strictly exclude any additional text or commentary. Strictly DO NOT add the title of the work at the beginning of the generated abstract. Strictly DO NOT add any instructions (like 'Here is the abstract..') at the beginning or end of the generated abstract. Strictly DO NOT add any extra text or notes at the beginning or end of the generated abstract. Strictly GIVE ONLY ONE paragraph as generated abstract.",NaN
104709,2002.08325v1,6564329,VQA-LOL: Visual Question Answering under the Lens of Logic,29,15,"Generate a concise 150-250 word abstract for the work titled 'VQA-LOL: Visual Question Answering under the Lens of Logic' in a single paragraph. Focus on describing the results, the problem addressed, and the key ideas and methodologies. Use formal academic and scientific language in the first-person plural and active voice. Only provide th

In [16]:
arxiv_papers2 = arxiv_papers2[~arxiv_papers2['Abstract'].isna()]

In [17]:
arxiv_papers2.shape[0]

3000

In [18]:
# Process DataFrame and save every 50 rows
for index, row in arxiv_papers2.iterrows():
    arxiv_papers2.at[index, 'machine_text'] = generate_text(row['Prompt'])
    if (index + 1) % save_count == 0:
        arxiv_papers2.to_csv(save_path, index=False)
        print(f'Saved progress at row {index + 1}')

# Save the final result
arxiv_papers2.to_csv(save_path, index=False)
print('Final save completed.')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Saved progress at row 50


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Saved progress at row 100


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Saved progress at row 2900


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Saved progress at row 2950


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Saved progress at row 3000
Final save completed.


In [19]:
import pandas as pd
import json

# Read the CSV file into a DataFrame
csv_file_path = '/content/drive/My Drive/arxiv_papers/arxiv_papers_with_MT.csv'  # Update this path to the location of your CSV file
df = pd.read_csv(csv_file_path)

# Function to convert each row to a JSON line
def row_to_json(row):
    return json.dumps(row.to_dict())

# Specify the path for the output JSONL file
jsonl_file_path = '/content/drive/My Drive/arxiv_papers/arxiv_papers_with_MT.jsonl'  # Update this path to the desired output location

# Write the JSONL file
with open(jsonl_file_path, 'w') as jsonl_file:
    for _, row in df.iterrows():
        jsonl_file.write(row_to_json(row) + '\n')

print(f"CSV file has been successfully converted to JSONL and saved at {jsonl_file_path}")


CSV file has been successfully converted to JSONL and saved at /content/drive/My Drive/arxiv_papers/arxiv_papers_with_MT.jsonl
